In [359]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pymysql
import datetime
plt.style.use('fivethirtyeight')
#plt.style.use('default')

from functools import reduce


pd.set_option('display.max_columns',None)
pd.set_option('display.min_rows', None)

In [360]:
df_account = pd.read_csv('df_account.csv') 
df_client = pd.read_csv('df_client.csv')
df_disp = pd.read_csv('df_disp.csv')
df_order = pd.read_csv('df_order.csv')
df_trans = pd.read_csv('df_trans.csv')
df_loan = pd.read_csv('df_loan.csv')
df_card = pd.read_csv('df_card.csv')
df_district = pd.read_csv('df_district.csv')

/Users/gthorani/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [361]:
df_order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,NaN


In [362]:
df_account.isnull().sum()*100/df_account.shape[0]

account_id     0.0
district_id    0.0
frequency      0.0
date           0.0
dtype: float64

In [363]:
df_account.dtypes

account_id      int64
district_id     int64
frequency      object
date           object
dtype: object

- "POPLATEK MESICNE" stands for monthly issuance
- "POPLATEK TYDNE" stands for weekly issuance
- "POPLATEK PO OBRATU" stands for issuance after transaction

In [364]:
# Let's convert to English 
df_account['frequency'].replace('POPLATEK MESICNE', 'monthly', inplace=True)
df_account['frequency'].replace('POPLATEK TYDNE', 'weekly', inplace=True)
df_account['frequency'].replace('POPLATEK PO OBRATU', 'after_trans', inplace=True)
df_account.rename(columns = {'frequency':'stmt_frq'}, inplace=True) #statement frequency

In [365]:
# Convert date to a datetime type variable
df_account['date'] = pd.to_datetime(df_account['date'])

In [366]:
df_client.head()

,client_id,gender,birth_date,district_id
0,1,F,1970-12-13,18
1,2,M,1945-02-04,1
2,3,F,1940-10-09,1
3,4,M,1956-12-01,5
4,5,F,1960-07-03,5


In [367]:
df_client.isnull().sum()*100/df_client.shape[0]

client_id      0.0
gender         0.0
birth_date     0.0
district_id    0.0
dtype: float64

In [368]:
df_client.dtypes

client_id       int64
gender         object
birth_date     object
district_id     int64
dtype: object

In [369]:
# Convert date to a datetime type variable
df_client['birth_date'] = pd.to_datetime(df_client['birth_date'])

In [370]:
df_disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [371]:
df_disp.dtypes

disp_id        int64
client_id      int64
account_id     int64
type          object
dtype: object

In [372]:
df_order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,NaN


In [373]:
df_order

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,NaN
5,29406,3,AB,59972357,3539.0,POJISTNE
6,29407,4,UV,26693541,2078.0,SIPO
7,29408,4,UV,5848086,1285.0,SIPO
8,29409,5,GH,37390208,2668.0,SIPO
9,29410,6,AB,44486999,3954.0,SIPO


In [374]:
df_order.isnull().sum()*100/df_order.shape[0]

order_id       0.000000
account_id     0.000000
bank_to        0.000000
account_to     0.000000
amount         0.000000
k_symbol      21.310462
dtype: float64

In [375]:
df_order.dtypes

order_id        int64
account_id      int64
bank_to        object
account_to      int64
amount        float64
k_symbol       object
dtype: object

- "POJISTNE" stands for insurrance payment
- "SIPO" stands for household
- "LEASING" stands for leasing
- "UVER" stands for loan payment

In [376]:
# Let's convert k_symbol to English
df_order['k_symbol'].replace('POJISTNE', 'insurance', inplace=True)
df_order['k_symbol'].replace('SIPO', 'household', inplace=True)
df_order['k_symbol'].replace('LEASING', 'leasing', inplace=True)
df_order['k_symbol'].replace('UVER', 'loan', inplace=True)
df_order['k_symbol'].replace(np.nan, 'unknown', inplace=True)
df_order.rename(columns = {'k_symbol':'order_payment_type'}, inplace=True) 

In [377]:
df_trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,NaN,NaN,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,NaN,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,NaN,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,NaN,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,NaN,AB,41403269.0


In [378]:
df_trans.isnull().sum()*100/df_trans.shape[0]

#bank & account have over 70% of rows missing - won't be using

trans_id       0.000000
account_id     0.000000
date           0.000000
type           0.000000
operation     17.335088
amount         0.000000
balance        0.000000
k_symbol      45.618847
bank          74.107467
account       72.036031
dtype: float64

In [379]:
df_trans['operation'].unique()

array(['VKLAD', 'PREVOD Z UCTU', 'PREVOD NA UCET', 'VYBER',
       'VYBER KARTOU', nan], dtype=object)

In [380]:
df_trans['k_symbol'].unique()

array([nan, 'SIPO', 'SLUZBY', 'UVER', ' ', 'POJISTNE', 'DUCHOD', 'UROK',
       'SANKC. UROK'], dtype=object)

In [381]:
df_trans.dtypes

trans_id        int64
account_id      int64
date           object
type           object
operation      object
amount          int64
balance         int64
k_symbol       object
bank           object
account       float64
dtype: object

In [382]:
# Convert date to a datetime type variable
df_trans['date'] = pd.to_datetime(df_trans['date'])

`type` - type of transaction
- "PRIJEM" stands for credit
- "VYDAJ" stands for withdrawal

`operation` - mode of transaction
- "VYBER KARTOU" credit card withdrawal
- "VKLAD" credit in cash
- "PREVOD Z UCTU" collection from another bank
- "VYBER" withdrawal in cash
- "PREVOD NA UCET" remittance to another bank

`k_symbol` - characterization of transaction
- "POJISTNE" stands for insurrance payment
- "SLUZBY" stands for payment for statement
- "UROK" stands for interest credited
- "SANKC. UROK" sanction interest if negative balance
- "SIPO" stands for household
- "DUCHOD" stands for old-age pension
- "UVER" stands for loan payment

In [383]:
# Replace names for type
df_trans['type'].replace('PRIJEM', 'credit', inplace=True)
df_trans['type'].replace('VYDAJ', 'withdrawal', inplace=True)
df_trans.rename(columns = {'type':'trans_type'}, inplace=True) 

# Replace names for operation
df_trans['operation'].replace('VYBER KARTOU', 'cc_withdrawal', inplace=True) #credit card withdrawal
df_trans['operation'].replace('VKLAD', 'c_cash', inplace=True) #credit in cash
df_trans['operation'].replace('PREVOD Z UCTU', 'col_bank', inplace=True) #collection from another bank
df_trans['operation'].replace('VYBER', 'withdrawal_c', inplace=True) #withdrawal in cash
df_trans['operation'].replace('PREVOD NA UCET', 'remittance', inplace=True) #withdrawal in cash
df_trans['operation'].replace(np.nan, 'unknown', inplace=True)

# Replace names for k_symbol
df_trans['k_symbol'].replace('POJISTNE', 'insurance', inplace=True) 
df_trans['k_symbol'].replace('SLUZBY', 'statement', inplace=True) 
df_trans['k_symbol'].replace('UROK', 'int_cred', inplace=True) #interest credited
df_trans['k_symbol'].replace('SANKC. UROK', 'sanc_int', inplace=True) #sanction interest if negative balance
df_trans['k_symbol'].replace('SIPO', 'household', inplace=True) 
df_trans['k_symbol'].replace('DUCHOD', 'pension', inplace=True) 
df_trans['k_symbol'].replace('UVER', 'loan', inplace=True) 
df_trans['k_symbol'].replace(np.nan, 'unknown', inplace=True)
df_trans['k_symbol'].replace(' ', 'unknown', inplace=True)
df_trans.rename(columns = {'k_symbol':'trans_payment_type'}, inplace=True) 

In [384]:
df_loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,1994-01-05,80952,24,3373.0,A
1,4961,19,1996-04-29,30276,12,2523.0,B
2,4962,25,1997-12-08,30276,12,2523.0,A
3,4967,37,1998-10-14,318480,60,5308.0,D
4,4968,38,1998-04-19,110736,48,2307.0,C


In [385]:
df_loan.isnull().sum()*100/df_loan.shape[0]

loan_id       0.0
account_id    0.0
date          0.0
amount        0.0
duration      0.0
payments      0.0
status        0.0
dtype: float64

In [386]:
df_loan.dtypes

loan_id         int64
account_id      int64
date           object
amount          int64
duration        int64
payments      float64
status         object
dtype: object

In [387]:
# Convert date to a datetime type variable
df_loan['date'] = pd.to_datetime(df_loan['date'])

In [388]:
# Rename loan columns
df_loan = df_loan.rename(columns={'amount': 'loan_amount', 'duration':'loan_duration', 'payments':'monthly_loan_payment', 'status':'loan_status'})

In [389]:
df_card.head()

,card_id,disp_id,type,issued
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [390]:
df_card.isnull().sum()*100/df_card.shape[0]

card_id    0.0
disp_id    0.0
type       0.0
issued     0.0
dtype: float64

In [391]:
df_card.dtypes

card_id     int64
disp_id     int64
type       object
issued     object
dtype: object

In [392]:
df_card['issued'] = pd.to_datetime(df_card['issued'])

In [393]:
df_district.head()

,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040


In [394]:
df_district.isnull().sum()*100/df_card.shape[0]

district_id    0.000000
A2             0.000000
A3             0.000000
A4             0.000000
A5             0.000000
A6             0.000000
A7             0.000000
A8             0.000000
A9             0.000000
A10            0.000000
A11            0.000000
A12            0.112108
A13            0.000000
A14            0.000000
A15            0.112108
A16            0.000000
dtype: float64

In [395]:
df_district.dtypes

district_id      int64
A2              object
A3              object
A4               int64
A5               int64
A6               int64
A7               int64
A8               int64
A9               int64
A10            float64
A11              int64
A12            float64
A13            float64
A14              int64
A15            float64
A16              int64
dtype: object

In [396]:
df_district = df_district.rename(columns={'A2':'district_name', 'A3':'region', 'A4':'population', 'A5':'nmu_lt499',
 'A6':'nmu_500to1999', 'A7':'nmu_2000to9999', 'A8':'nmu_gt10000',
 'A9':'n_cty', 'A10':'ratio_urban', 'A11':'avg_salary', 'A12':'unemp_95', 'A13': 'unemp_96',
 'A14':'nentrep_p1000', 'A15':'ncrimes_95', 'A16':'ncrimes_96'})

In [397]:
# Time to merge different dateframes

df_final = pd.merge(df_account, df_disp, on = 'account_id') #shape: 5369, 7

In [398]:
df_final = pd.merge(df_final, df_district, on = 'district_id') # shape: 5369, 22

In [399]:
df_final = pd.merge(df_final, df_client, on = 'client_id') #shape: 5369, 25

#can drop one of the district_id

In [400]:
df_final = pd.merge(df_final, df_card, on='disp_id', how='outer', suffixes=('_disp','_card')) #shape: 5369, 28

In [401]:
df_final = pd.merge(df_final, df_loan, on='account_id', how='inner', suffixes=('_account','_loan')) #shape: 827, 34

#We only keep the accounts with loans 

In [402]:
len(df_final['client_id'].unique()) 

# There are 827 unique clients with loans

827

In [403]:
len(df_final['account_id'].unique()) 

# This makes sense as the semi anonymized dataset contains 606 successful and 76 not successful loans.

682

In [404]:
df_final.head()

,account_id,district_id_x,stmt_frq,date_account,disp_id,client_id,type_disp,district_name,region,population,nmu_lt499,nmu_500to1999,nmu_2000to9999,nmu_gt10000,n_cty,ratio_urban,avg_salary,unemp_95,unemp_96,nentrep_p1000,ncrimes_95,ncrimes_96,gender,birth_date,district_id_y,card_id,type_card,issued,loan_id,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status
0,2350,18,monthly,1996-08-04,2841,2841,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1973-04-10,18,446.0,classic,1998-06-04,5451,1997-05-11,159744,48,3328.0,C
1,9156,18,monthly,1997-09-14,10963,11271,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,M,1937-12-08,18,NaN,NaN,NaT,6856,1998-12-01,163332,36,4537.0,C
2,10973,18,weekly,1993-04-20,13182,13490,OWNER,Pisek,south Bohemia,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,F,1969-05-25,18,NaN,NaN,NaT,7235,1993-10-13,154416,48,3217.0,A
3,2,1,monthly,1993-02-26,2,2,OWNER,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,M,1945-02-04,1,NaN,NaN,NaT,4959,1994-01-05,80952,24,3373.0,A
4,2,1,monthly,1993-02-26,3,3,DISPONENT,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,F,1940-10-09,1,NaN,NaN,NaT,4959,1994-01-05,80952,24,3373.0,A


In [405]:
df_trans_account = pd.merge(df_trans, df_final[['account_id','date_loan','loan_amount','loan_duration','monthly_loan_payment','loan_status']], on = 'account_id') 

#shape: 233627, 11 transactions for accounts with loans

In [406]:
df_trans_account.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status
0,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05,80952,24,3373.0,A
1,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05,80952,24,3373.0,A
2,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A
3,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A
4,280,2,1993-04-12,credit,col_bank,20236,45286,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A


In [407]:
df_trans['trans_type'].unique()

array(['credit', 'withdrawal', 'VYBER'], dtype=object)

In [408]:
# Get the difference between the date of the loan and the date of the transaction
df_trans_account['date_diff'] = (df_trans_account['date_loan'] - df_trans_account['date']) 

In [513]:
# Let's drop transactions that occured after the loan date
df_trans_account.drop(df_trans_account[df_trans_account['date_diff'] < datetime.timedelta(0)].index, inplace=True) 

In [514]:
df_trans_account.shape

(54860, 17)

In [517]:
df_trans_account.sort_values(by='date',ascending=False)

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
173289,2616301,8645,1998-12-05,withdrawal,remittance,8919,19814,household,KL,37808364.0,1998-12-08,240900,60,4015.0,C,3 days,0
127325,2042344,6922,1998-12-03,withdrawal,withdrawal_c,25500,47956,unknown,NaN,NaN,1998-12-06,139488,24,5812.0,C,3 days,0
2950,32739,105,1998-12-02,credit,c_cash,12100,35875,unknown,NaN,NaN,1998-12-05,352704,48,7348.0,C,3 days,0
2925,32697,105,1998-12-02,withdrawal,withdrawal_c,5700,30175,unknown,NaN,NaN,1998-12-05,352704,48,7348.0,C,3 days,0
31076,566488,1928,1998-11-30,withdrawal,withdrawal_c,15,38449,statement,NaN,NaN,1998-12-06,55632,24,2318.0,C,6 days,0
21659,377228,1284,1998-11-30,withdrawal,withdrawal_c,15,41035,statement,NaN,NaN,1998-12-05,52512,12,4376.0,C,5 days,0
127306,2042325,6922,1998-11-30,withdrawal,withdrawal_c,30,73456,statement,NaN,NaN,1998-12-06,139488,24,5812.0,C,6 days,0
173381,3502674,8645,1998-11-30,credit,unknown,106,28747,int_cred,NaN,NaN,1998-12-08,240900,60,4015.0,C,8 days,0
40184,662795,2262,1998-11-30,withdrawal,withdrawal_c,15,94157,statement,NaN,NaN,1998-12-02,230220,36,6395.0,C,2 days,0
182995,2767462,9156,1998-11-30,withdrawal,withdrawal_c,15,17621,statement,NaN,NaN,1998-12-01,163332,36,4537.0,C,1 days,0


In [411]:
df_trans_account['target'] = df_trans_account['loan_status']

In [412]:
# Let's convert the status (A, B, C, D) to target (0,1)
df_trans_account.loc[df_trans_account['target'].isin(['A','C']),'target'] = 0  # Accepted loans
df_trans_account.loc[df_trans_account['target'].isin(['B','D']),'target'] = 1  # Rejected loans

In [413]:
#Remove duplicate transactions 
df_trans_account = df_trans_account.drop_duplicates()

In [518]:
df_trans_account.shape

(54860, 17)

In [415]:
df_trans_account.groupby('trans_payment_type').count()

,trans_id,account_id,date,trans_type,operation,amount,balance,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
trans_payment_type,,,,,,,,,,,,,,,,
household,16328,16328,16328,16328,16328,16328,16328,15262,16328,16328,16328,16328,16328,16328,16328,16328
insurance,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891,3891
int_cred,31338,31338,31338,31338,31338,31338,31338,0,0,31338,31338,31338,31338,31338,31338,31338
loan,13580,13580,13580,13580,13580,13580,13580,13579,13579,13580,13580,13580,13580,13580,13580,13580
sanc_int,589,589,589,589,589,589,589,0,0,589,589,589,589,589,589,589
statement,23708,23708,23708,23708,23708,23708,23708,0,0,23708,23708,23708,23708,23708,23708,23708
unknown,102122,102122,102122,102122,102122,102122,102122,17781,28827,102122,102122,102122,102122,102122,102122,102122


In [416]:
df_trans_account.groupby('operation').count()

,trans_id,account_id,date,trans_type,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
operation,,,,,,,,,,,,,,,,
c_cash,29001,29001,29001,29001,29001,29001,29001,0,0,29001,29001,29001,29001,29001,29001,29001
cc_withdrawal,965,965,965,965,965,965,965,0,965,965,965,965,965,965,965,965
col_bank,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106,8106
remittance,42408,42408,42408,42408,42408,42408,42408,42407,42407,42408,42408,42408,42408,42408,42408,42408
unknown,31338,31338,31338,31338,31338,31338,31338,0,0,31338,31338,31338,31338,31338,31338,31338
withdrawal_c,79738,79738,79738,79738,79738,79738,79738,0,11147,79738,79738,79738,79738,79738,79738,79738


In [417]:
df_trans_account_sanc = df_trans_account[df_trans_account['trans_payment_type']=='sanc_int']

In [418]:
df_trans_account_sanc.groupby('loan_status')['amount'].sum()

loan_status
A     299
B    3103
C      58
D    5986
Name: amount, dtype: int64

In [419]:
df_trans_account_sanc.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
1174,3445015,19,1996-04-30,withdrawal,withdrawal_c,1,15958,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-1 days,1
1196,3445037,19,1998-01-31,withdrawal,withdrawal_c,15,7138,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-642 days,1
1205,3445046,19,1998-09-30,withdrawal,withdrawal_c,7,7507,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-884 days,1
1208,3445049,19,1998-11-30,withdrawal,withdrawal_c,5,8211,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-945 days,1
1223,3681181,19,1996-04-30,withdrawal,withdrawal_c,1,15942,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-1 days,1


In [420]:
df_trans_account_intcred = df_trans_account[df_trans_account['trans_payment_type']=='int_cred']

In [421]:
df_trans_account_intcred.groupby('loan_status')['amount'].sum()

loan_status
A    2224168
B     271661
C    2522274
D     284182
Name: amount, dtype: int64

In [422]:
df_trans_account_intcred.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
816,3530483,2,1993-03-31,credit,unknown,14,25050,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,280 days,0
818,3530484,2,1993-04-30,credit,unknown,110,34395,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,250 days,0
820,3530485,2,1993-05-31,credit,unknown,145,37176,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,219 days,0
822,3530486,2,1993-06-30,credit,unknown,160,45290,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,189 days,0
824,3530487,2,1993-07-31,credit,unknown,203,39084,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,158 days,0


## Remittance Transactions

In [423]:
df_trans_account_rem = df_trans_account[df_trans_account['operation']=='remittance']

In [424]:
df_trans_account_rem.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
154,361,2,1994-01-12,withdrawal,remittance,3373,35338,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-7 days,0
156,362,2,1994-02-12,withdrawal,remittance,3373,29636,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-38 days,0
158,363,2,1994-03-12,withdrawal,remittance,3373,35827,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-66 days,0
160,364,2,1994-04-12,withdrawal,remittance,3373,40639,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-97 days,0
162,365,2,1994-05-12,withdrawal,remittance,3373,42772,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-127 days,0


In [425]:
df_trans_account_rem.groupby(['target'])['balance'].mean()

target
0    41807.397356
1    30131.767407
Name: balance, dtype: float64

In [426]:
df_trans_account_rem.sort_values(by='loan_amount', ascending=False)

#590820 is the highest loan amount for rem. trans.
#4980 is the lowest loan amount for rem. trans.

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
138207,2286756,7542,1998-02-05,withdrawal,remittance,3389,46000,unknown,UV,94022273.0,1997-10-19,590820,60,9847.0,C,-109 days,0
138180,2286719,7542,1998-01-12,withdrawal,remittance,3683,32467,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-85 days,0
138192,2286741,7542,1996-11-05,withdrawal,remittance,3389,28331,unknown,UV,94022273.0,1997-10-19,590820,60,9847.0,C,348 days,0
138191,2286730,7542,1998-12-12,withdrawal,remittance,3683,58397,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-419 days,0
138190,2286729,7542,1998-11-12,withdrawal,remittance,3683,56384,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-389 days,0
138189,2286728,7542,1998-10-12,withdrawal,remittance,3683,55075,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-358 days,0
138188,2286727,7542,1998-09-12,withdrawal,remittance,3683,54665,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-328 days,0
138187,2286726,7542,1998-08-12,withdrawal,remittance,3683,47500,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-297 days,0
138186,2286725,7542,1998-07-12,withdrawal,remittance,3683,59022,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-266 days,0
138185,2286724,7542,1998-06-12,withdrawal,remittance,3683,40869,household,OP,63780206.0,1997-10-19,590820,60,9847.0,C,-236 days,0


In [427]:
df_trans_account_rem.groupby(['target'])['loan_amount'].sum()

target
0    5333000604
1     770792172
Name: loan_amount, dtype: int64

In [428]:
# 191556 transactions are remittance 
df_trans_account_rem['target'].value_counts(normalize=True)

0    0.920416
1    0.079584
Name: target, dtype: float64

In [429]:
df_rem_count = df_trans_account_rem.groupby('account_id')['amount'].count().to_frame()

In [430]:
df_rem_mean = df_trans_account_rem.groupby('account_id')['amount'].mean().to_frame()

In [431]:
df_trans_account_rem = pd.merge(df_trans_account_rem, df_rem_count, on = 'account_id') 

In [432]:
df_trans_account_rem = pd.merge(df_trans_account_rem, df_rem_mean, on = 'account_id') 

In [486]:
df_trans_account_rem.head()

,trans_id,account_id,date,trans_type,operation,amount_x,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target,amount_y,amount
0,361,2,1994-01-12,withdrawal,remittance,3373,35338,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-7 days,0,89,6216.202247
1,362,2,1994-02-12,withdrawal,remittance,3373,29636,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-38 days,0,89,6216.202247
2,363,2,1994-03-12,withdrawal,remittance,3373,35827,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-66 days,0,89,6216.202247
3,364,2,1994-04-12,withdrawal,remittance,3373,40639,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-97 days,0,89,6216.202247
4,365,2,1994-05-12,withdrawal,remittance,3373,42772,loan,ST,89597016.0,1994-01-05,80952,24,3373.0,A,-127 days,0,89,6216.202247


In [473]:
df_trans_account_rem.groupby(['target'])['amount_x'].sum() #trans. amount

target
0    153440916
1     13754572
Name: amount_x, dtype: int64

In [481]:
df_trans_account_rem.groupby('target')['amount_y'].mean() # avg. num. of trans.

target
0    110.856839
1     96.134222
Name: amount_y, dtype: float64

In [436]:
df_trans_account_rem.groupby('target')['amount'].mean() #avg. trans. amount

target
0    3931.056183
1    4075.428741
Name: amount, dtype: float64

In [488]:
df_trans_account_rem['trans_payment_type'].unique()

array(['loan', 'household', 'unknown', 'insurance'], dtype=object)

## Sanction interest if negative balance transactions (default interest)

In [455]:
df_trans_account.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
0,276,2,1993-02-26,credit,c_cash,1100,1100,unknown,NaN,NaN,1994-01-05,80952,24,3373.0,A,313 days,0
2,279,2,1993-03-12,credit,col_bank,20236,21336,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A,299 days,0
4,280,2,1993-04-12,credit,col_bank,20236,45286,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A,268 days,0
6,281,2,1993-05-12,credit,col_bank,20236,54631,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A,238 days,0
8,282,2,1993-06-12,credit,col_bank,30354,67530,unknown,ST,66487163.0,1994-01-05,80952,24,3373.0,A,207 days,0


In [456]:
df_trans_account['trans_payment_type'].unique()

array(['unknown', 'loan', 'household', 'statement', 'int_cred',
       'sanc_int', 'insurance'], dtype=object)

In [457]:
df_trans_account_sanc = df_trans_account[df_trans_account['trans_payment_type']=='sanc_int']

In [458]:
df_trans_account_sanc.sample(5)

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
220499,3458363,10788,1996-06-30,withdrawal,withdrawal_c,10,703,sanc_int,NaN,NaN,1994-08-23,45768,12,3814.0,A,-677 days,0
147017,3492876,7824,1996-11-30,withdrawal,withdrawal_c,11,-2455,sanc_int,NaN,NaN,1994-09-28,189696,24,7904.0,B,-794 days,1
66313,3449372,3678,1998-11-30,withdrawal,withdrawal_c,38,-7421,sanc_int,NaN,NaN,1995-02-05,316140,60,5269.0,D,-1394 days,1
9479,3545165,442,1998-12-31,withdrawal,withdrawal_c,1,54377,sanc_int,NaN,NaN,1997-07-10,357840,48,7455.0,D,-539 days,1
13023,3681279,808,1996-03-31,withdrawal,withdrawal_c,7,-3379,sanc_int,NaN,NaN,1996-04-11,215616,48,4492.0,D,11 days,1


In [459]:
df_trans_account_sanc.groupby(['target'])['balance'].mean() #avg balance by target

target
0    20496.953125
1     6245.550476
Name: balance, dtype: float64

In [460]:
df_trans_account_sanc.sort_values(by='loan_amount', ascending=True)

#541200 is the highest loan amount for sanc. trans.
#29448 is the lowest loan amount for sanc. trans.

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
184773,3509025,9199,1998-07-31,withdrawal,withdrawal_c,9,-335,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-688 days,1
184713,3456411,9199,1997-01-31,withdrawal,withdrawal_c,7,-1703,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-142 days,1
184715,3456413,9199,1997-02-28,withdrawal,withdrawal_c,6,14441,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-170 days,1
184723,3456421,9199,1997-09-30,withdrawal,withdrawal_c,3,277,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-384 days,1
184725,3456423,9199,1997-10-31,withdrawal,withdrawal_c,1,5417,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-415 days,1
184729,3456427,9199,1998-01-31,withdrawal,withdrawal_c,5,-2356,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-507 days,1
184767,3509019,9199,1998-02-28,withdrawal,withdrawal_c,6,14341,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-535 days,1
184737,3456435,9199,1998-07-31,withdrawal,withdrawal_c,9,-359,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-688 days,1
184739,3456437,9199,1998-08-31,withdrawal,withdrawal_c,2,12846,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-719 days,1
184751,3509003,9199,1997-01-31,withdrawal,withdrawal_c,6,-1709,sanc_int,NaN,NaN,1996-09-11,29448,12,2454.0,B,-142 days,1


In [461]:
#569 transactions are sanc.
df_trans_account_sanc['target'].value_counts() 

1    525
0     64
Name: target, dtype: int64

In [462]:
df_trans_account_sanc['target'].value_counts(normalize=True)

1    0.891341
0    0.108659
Name: target, dtype: float64

In [463]:
df_trans_account_sanc.groupby(['target'])['loan_amount'].sum() #total loan amount per target

target
0      5847432
1    107417508
Name: loan_amount, dtype: int64

In [464]:
#CHECK how much money is still owing ???

In [465]:
df_sanc_count = df_trans_account_sanc.groupby('account_id')['amount'].count().to_frame()
df_sanc_mean = df_trans_account_sanc.groupby('account_id')['amount'].mean().to_frame()

In [466]:
df_trans_account_sanc = pd.merge(df_trans_account_sanc, df_sanc_count, on = 'account_id') 
df_trans_account_sanc = pd.merge(df_trans_account_sanc, df_sanc_mean, on = 'account_id') 

In [490]:
df_trans_account_sanc.head()

,trans_id,account_id,date,trans_type,operation,amount_x,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target,amount_y,amount
0,3445015,19,1996-04-30,withdrawal,withdrawal_c,1,15958,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-1 days,1,8,7.25
1,3445037,19,1998-01-31,withdrawal,withdrawal_c,15,7138,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-642 days,1,8,7.25
2,3445046,19,1998-09-30,withdrawal,withdrawal_c,7,7507,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-884 days,1,8,7.25
3,3445049,19,1998-11-30,withdrawal,withdrawal_c,5,8211,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-945 days,1,8,7.25
4,3681181,19,1996-04-30,withdrawal,withdrawal_c,1,15942,sanc_int,NaN,NaN,1996-04-29,30276,12,2523.0,B,-1 days,1,8,7.25


In [468]:
# all transactions are withdrawals in cash
df_trans_account_sanc['operation'].unique()

array(['withdrawal_c'], dtype=object)

In [474]:
df_trans_account_sanc.groupby('target')['amount_x'].sum() # amount of a trans. by target

target
0     357
1    9089
Name: amount_x, dtype: int64

In [470]:
df_trans_account_sanc.groupby('target')['amount_y'].mean() # num. of trans.

target
0    12.093750
1    15.007619
Name: amount_y, dtype: float64

In [471]:
df_trans_account_sanc.groupby('target')['amount'].mean() # avg trans. amount 

target
0     5.578125
1    17.312381
Name: amount, dtype: float64

## Interest Credited

In [493]:
df_trans_account_int = df_trans_account[df_trans_account['trans_payment_type']=='int_cred']

In [494]:
df_trans_account_int.head()

,trans_id,account_id,date,trans_type,operation,amount,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target
816,3530483,2,1993-03-31,credit,unknown,14,25050,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,280 days,0
818,3530484,2,1993-04-30,credit,unknown,110,34395,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,250 days,0
820,3530485,2,1993-05-31,credit,unknown,145,37176,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,219 days,0
822,3530486,2,1993-06-30,credit,unknown,160,45290,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,189 days,0
824,3530487,2,1993-07-31,credit,unknown,203,39084,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,158 days,0


In [496]:
df_trans_account_int.shape

(31338, 17)

In [495]:
df_trans_account_int['target'].value_counts()

0    26887
1     4451
Name: target, dtype: int64

In [501]:
df_trans_account_int.groupby(['target'])['balance'].mean() #avg balance by target

target
0    43287.954476
1    32334.766569
Name: balance, dtype: float64

In [502]:
df_int_count = df_trans_account_int.groupby('account_id')['amount'].count().to_frame()
df_int_mean = df_trans_account_int.groupby('account_id')['amount'].mean().to_frame()

df_trans_account_int = pd.merge(df_trans_account_int, df_int_count, on = 'account_id') 
df_trans_account_int = pd.merge(df_trans_account_int, df_int_mean, on = 'account_id') 

In [503]:
df_trans_account_int.head()

,trans_id,account_id,date,trans_type,operation,amount_x,balance,trans_payment_type,bank,account,date_loan,loan_amount,loan_duration,monthly_loan_payment,loan_status,date_diff,target,amount_y,amount
0,3530483,2,1993-03-31,credit,unknown,14,25050,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,280 days,0,70,153.128571
1,3530484,2,1993-04-30,credit,unknown,110,34395,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,250 days,0,70,153.128571
2,3530485,2,1993-05-31,credit,unknown,145,37176,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,219 days,0,70,153.128571
3,3530486,2,1993-06-30,credit,unknown,160,45290,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,189 days,0,70,153.128571
4,3530487,2,1993-07-31,credit,unknown,203,39084,int_cred,NaN,NaN,1994-01-05,80952,24,3373.0,A,158 days,0,70,153.128571


In [505]:
df_trans_account_int.groupby('target')['amount_x'].sum() # amount of a trans. by target

target
0    4746442
1     555843
Name: amount_x, dtype: int64

In [506]:
df_trans_account_int.groupby('target')['amount_y'].mean() # num. of trans.

target
0    59.261985
1    73.545495
Name: amount_y, dtype: float64

In [507]:
df_trans_account_int.groupby('target')['amount'].mean() # avg trans. amount 

target
0    176.532971
1    124.880476
Name: amount, dtype: float64